# 1. Imports


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import glob
import torch
import torch.nn as nn
from pathlib import Path

import re
import string
import math
from collections import defaultdict
import random
import warnings
from tqdm import tqdm
from nltk.corpus import stopwords
from prettytable import PrettyTable

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc,accuracy_score,auc,accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import AdamW
from transformers import TrainingArguments, EarlyStoppingCallback
from transformers import Trainer
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer


import torch.nn.utils.prune as prune
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Linear, Module

import wandb
import optuna
#from prettytable import PrettyTable
import copy
from datasets import Dataset,DatasetDict
from datasets import load_dataset, load_metric ,concatenate_datasets


In [2]:
data_path = 'data/'
tb_dir = 'tbs_HW2/'
results = 'results/'
models_trained_params = 'models_trained_params/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
home_dir = os.getcwd()

# 2. Preparing the data

Fetching the data

In [3]:
def get_clean_data(name):
    data = pd.read_csv(Path(home_dir, name)).iloc[:,1:].rename({'filtered_text_2':'filtered_text'},axis=1)
#     clean_data=clean_data(data) #replace with the real funcion name
    return data


data=get_clean_data('preprocessed_df.csv')


Encoding the labels

In [4]:
features=data['filtered_text']
labels=data['main_category'].values


# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder on the labels and transform the labels
encoded_labels = label_encoder.fit_transform(labels)

# Replace the labels column in the dataframe with the encoded labels
data['encoded_main_category'] = encoded_labels

Splitting the data

In [5]:
def split_data(x, y, test_ratio=0.2, random_state=42 ,batch_size=32):
    train_size=int((1-test_ratio)*(x.shape[0]))
    test_size=int((x.shape[0])-train_size)

    train_end_idx = train_size
    test_end_idx = train_size + test_size
    # Make one list for all the reviews
    headlines = x.tolist()

    # Mini sample of reviews for train and test
    train_data = headlines[:train_end_idx]
    test_data = headlines[train_end_idx:test_end_idx]
    # Take mini sample of the labels and preprocess them such that we can use them in the model training loop
    labels = encoded_labels.tolist()
    train_labels = labels[:train_end_idx]
    test_labels = labels[train_end_idx:test_end_idx]

    # Use data set class in order to build train and test datasets
    train_df = pd.DataFrame({'text':train_data, 'labels':train_labels})#.dropna().drop_duplicates()
    test_df = pd.DataFrame({'text':test_data, 'labels':test_labels})#.dropna().drop_duplicates()
    train_df.to_csv('train_df.csv', index = False)
    test_df.to_csv('test_df.csv', index = False)
    
    data_files = {
    'train':'train_df.csv',
    'test':'test_df.csv'
    }
    
    raw_datasets = load_dataset("csv", data_files=data_files)
    

    return raw_datasets


In [6]:
dataset = split_data(features, labels)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/liyag/.cache/huggingface/datasets/csv/default-2e78edd390e366fa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 388761
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 97191
    })
})

dataset

# Knowladge distilation 

In [8]:
def knowladge_distilation_training(student_model, teacher_model, train_dataset):
    epochs = 10
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=1e-5)
    softmax = torch.nn.Softmax()
    relu = torch.nn.ReLU()
    # Knowledge distillation training loop
    for epoch in range(epochs):
        for batch in train_dataset:
            # Forward pass with the teacher model to generate soft targets
            input_ids = batch['input_ids']
            input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension
            attention_mask = batch['attention_mask']
            attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)  # Add batch dimension

            with torch.no_grad():
                teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits

            # Forward pass with the student model
            student_outputs = student_model(input_ids, attention_mask=attention_mask)
            student_logits = student_outputs.logits

            # Compute the distillation loss
            temperature = 2  # Temperature parameter for softening the logits
            soft_teacher_logits = softmax(teacher_logits / temperature)
            soft_student_logits = softmax(student_logits / temperature)
            distillation_loss = torch.nn.KLDivLoss()(torch.log_softmax(student_logits, dim=-1),
                                                     soft_teacher_logits.detach())

            # Optionally, add other losses such as cross-entropy or task-specific losses

            # Backpropagation and optimization
            optimizer.zero_grad()
            total_loss = distillation_loss  # You can add other losses here
            total_loss.backward()
            optimizer.step()

    # Optionally, you can fine-tune the student model on labeled data using traditional supervised learning techniques
    return student_model

# 3. Creating the model class

In [17]:
class OurAwesomeModel():
    def __init__(self,model_name,dataset):
        self.model_name=model_name
        self.model=AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=6,return_dict=True).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.dataset=dataset
        self.token_args={"max_length": 32, "truncation": True, "padding": "max_length"}
        
        self.tokenized_datasets = self.tokenization()
        
    # function that tokenize the data and returns train and test tokenized dataset in torch format
    def tokenization(self):
        tokenized_dataset=self.dataset.map(self.tokenizer, input_columns='text', fn_kwargs=self.token_args)
        tokenized_dataset.set_format('torch')
        return tokenized_dataset
    
    # function that defines the Trainer object and train the model
    def train(self,train_args):
        train_args = TrainingArguments(**train_args)
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_datasets['train'],
            eval_dataset=self.tokenized_datasets['test'],
            compute_metrics=self.metric_fn)
        trainer.train()

    
    # definition of the metric we want to optimize
    def metric_fn(self, predictions):
        preds = predictions.predictions.argmax(axis=1)
        labels = predictions.label_ids
        return {'f1': f1_score(labels, preds, average='weighted')
               ,'accuracy':accuracy_score(labels,preds)}
    
    # help function for hyperparameter finetuning
    def model_init(self):
        return self.model
    
    # function that prefurms hyperparameter finetuning with optuna as backend. 
    # The function gets the train arguments from the function 'my_hp_space'.
    def hpm_search(self,train_args_dict):
        wandb.init(project="model1", name = self.model_name)
        train_args = TrainingArguments(**train_args_dict)
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_datasets['train'],
            eval_dataset=self.tokenized_datasets['test'],
            model_init=self.model_init,
            compute_metrics=self.metric_fn)
        
        best_run=trainer.hyperparameter_search(direction="maximize", hp_space=self.my_hp_space,n_trials=10)
        best_hyperparameters = best_run.hyperparameters
        print(best_hyperparameters)
        train_args_dict.update(best_hyperparameters)
        print(train_args_dict)
        self.train(TrainingArguments(**train_args_dict))
        self.save_model()
        wandb.finish()
    
    # function that is called in every optuna trial, and return a combination of hyperparameters to try. 
    def my_hp_space(self,trial):
        return {"learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True),
                "num_train_epochs": trial.suggest_categorical("num_train_epochs", [10]),
                "seed": trial.suggest_categorical("seed", [0]),
                "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16,32]),
                "gradient_accumulation_steps":trial.suggest_int("gradient_accumulation_steps",1,6),
                "warmup_steps":trial.suggest_int("warmup_steps",0,500),
                "weight_decay":trial.suggest_float("weight_decay",1e-4,1e-2),
                "per_device_eval_batch_size":trial.suggest_categorical("per_device_eval_batch_size",[16,32])}
    
    
    def evaluate(self, train_args_dict):
        train_args = TrainingArguments(**train_args_dict)
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_datasets['train'],
            eval_dataset=self.tokenized_datasets['test'],
            compute_metrics=self.metric_fn)
        
        predictions = trainer.predict(self.tokenized_datasets['test'])
        result_dict = self.metric_fn(predictions)
        for k,v in result_dict.items():
            print(f'{k} value: {v}')
    
    # First contraction function
    # pruns the amount% of the smallest weights
    def pruning(self,amount):           
        for module_name, module in self.model.named_modules():
            if isinstance(module, Linear):# and 'ff' in module_name:
                print(f'\n{module_name}:\nold_total_weights = {module.weight.sum()}')
                prune.l1_unstructured(module, name='weight', amount=amount)
                prune.remove(module, name='weight')
                print(f'new_total_weights = {module.weight.sum()}')

    # second contraction function
    # saves the weights in half of the space 
    def half(self):
        self.model = self.model.half()
    
    # function that prints the models' weights. 
    # The weights are presented in their absolute value such that negative and positive weights wont offset
    def show_parameters(self):
        table = PrettyTable(["Modules", "Parameters","Sum of Tensor"])
        total_params = 0
        total_sum = 0
        for name, parameter in self.model.named_parameters():
            if not parameter.requires_grad: 
                continue
            params = parameter.numel()
            total_sum += parameter.abs().sum()
            total=float(str(parameter.abs().sum()).split(',')[0][7:])
            table.add_row([name, params,total])
            total_params+=params
        table.add_row(["Total Trainable Params",str(total_params),str(total_sum.item())])
        print(table)
        return total_params

    def save_model(self, model_type = None):
        if model_type == None:
            torch.save(self.model.state_dict(), f'{self.model_name}.pt')
        else:
            torch.save(self.model.state_dict(), f'{model_type}_{self.model_name}.pt')
        print('model was saved')
        
    def load_trained_model(self, model_type = None):
        if model_type == None:
            self.model.load_state_dict(torch.load(f'{self.model_name}.pt'))
        else:
            self.model.load_state_dict(torch.load(f'{model_type}_{self.model_name}.pt'))
        print('model was loaded')

# 4 Model initialization

Intializing params

In [10]:
wandb.finish()

In [11]:
wandb.init(project="model1")

OUT_PATH=Path(home_dir,'results')

args = {'output_dir':OUT_PATH,
 'overwrite_output_dir':True,
 'greater_is_better':True,
 'evaluation_strategy':'steps',
 'do_train':True,
 'logging_strategy':'epoch',
 'save_strategy':'epoch',
 'report_to':'wandb'}

wandb: Currently logged in as: liyag (delta_lxr). Use `wandb login --relogin` to force relogin


In [13]:
model_names = ["xlnet-base-cased","bert-base-uncased"]
for model_name in model_names:
    #Initializing the model
    print('==========================================================================')
    print(f'============================== {model_name} ==============================')
    print('==========================================================================')
    this_model=OurAwesomeModel(model_name,dataset)
    this_model.show_parameters()
    this_model.evaluate(args)
    #finding the best hyperparameters and retraining the model
    print(f'------------------------------ regular {model_name} ------------------------------')
    this_model.hpm_search(args)
    this_model.show_parameters()
    this_model.evaluate(args)

============================== xlnet-base-cased ==============================


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Map:   0%|          | 0/388761 [00:00<?, ? examples/s]

Map:   0%|          | 0/97191 [00:00<?, ? examples/s]

+-------------------------------------------------+------------+---------------+
|                     Modules                     | Parameters | Sum of Tensor |
+-------------------------------------------------+------------+---------------+
|               transformer.mask_emb              |    768     |     7.254     |
|        transformer.word_embedding.weight        |  24576000  |   1074791.5   |
|          transformer.layer.0.rel_attn.q         |   589824   |   44567.0234  |
|          transformer.layer.0.rel_attn.k         |   589824   |   94692.3594  |
|          transformer.layer.0.rel_attn.v         |   589824   |   15121.5391  |
|          transformer.layer.0.rel_attn.o         |   589824   |   16153.6406  |
|          transformer.layer.0.rel_attn.r         |   589824   |   10870.3428  |
|      transformer.layer.0.rel_attn.r_r_bias      |    768     |    119.4765   |
|      transformer.layer.0.rel_attn.r_s_bias      |    768     |    135.992    |
|      transformer.layer.0.r

...

Step,Training Loss,Validation Loss,F1,Accuracy
500,No log,2.146050,0.712637,0.714336
1000,No log,2.507567,0.707754,0.707267
1500,No log,2.476681,0.707443,0.709027
2000,No log,1.485119,0.697412,0.698377
2500,No log,1.436344,0.702620,0.704942
3000,No log,1.480121,0.712002,0.711949
3500,No log,1.337037,0.699879,0.698871
4000,No log,1.428410,0.702242,0.701372
4500,No log,1.372612,0.705264,0.705281
5000,No log,1.395945,0.702004,0.702503


model was saved


eval/accuracy,▄▄▃▃▁▄▁▂▂▃▂▃▃▃▃▄▃▃▄▄▃▄▄▅▄▄▄▅▅▅▅▆▆▆▇▇▇███
eval/f1,▄▄▃▃▁▄▁▂▃▃▂▂▃▃▃▄▃▃▄▄▃▄▄▅▄▄▄▅▆▅▅▆▆▆▇▇████
eval/loss,▆▇██▄▄▃▄▃▃▄▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
eval/runtime,▄▆▄▇▂▃▅▅▃▁▇▁▇▄▆▁▇▂▄▅▆▃▂▇▃▇▄▇▂▇▃▅▅█▂▂█▆▆▃
eval/samples_per_second,▅▃▅▂▇▆▄▄▆█▂█▂▅▃█▂▇▅▄▃▆▇▂▆▂▅▂▇▂▆▄▄▁▇▇▁▃▃▆
eval/steps_per_second,▅▃▅▂▇▆▄▄▆█▂█▂▅▃█▂▇▅▄▃▆▇▂▆▂▅▂▇▂▆▄▄▁▇▇▁▃▃▆
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁
train/loss,▁
train/total_flos,▁


+---------------------------------------------------------+------------+---------------+
|                         Modules                         | Parameters | Sum of Tensor |
+---------------------------------------------------------+------------+---------------+
|          bert.embeddings.word_embeddings.weight         |  23440896  |  955872.1875  |
|        bert.embeddings.position_embeddings.weight       |   393216   |    4577.082   |
|       bert.embeddings.token_type_embeddings.weight      |    1536    |    13.9256    |
|             bert.embeddings.LayerNorm.weight            |    768     |    652.7019   |
|              bert.embeddings.LayerNorm.bias             |    768     |    31.7363    |
|     bert.encoder.layer.0.attention.self.query.weight    |   589824   |   20028.5293  |
|      bert.encoder.layer.0.attention.self.query.bias     |    768     |    167.1584   |
|      bert.encoder.layer.0.attention.self.key.weight     |   589824   |   19688.5469  |
|       bert.encoder.

f1 value: 0.733591514935646
accuracy value: 0.734841703449908


Training loop

In [43]:
model_names = ["xlnet-base-cased","bert-base-uncased"]
models={}

for model_name in model_names:
    #Initializing the model
    print('=============================================================================')
    print(f'============================== {model_name} ==============================')
    print('=============================================================================')
    this_model=OurAwesomeModel(model_name,dataset)
    print(f'------------------------------ regular {model_name} ------------------------------')
    # load the best parameters of the model
    this_model.load_trained_model()
    print(f'------------------------------ {model_name} weights ------------------------------')
    this_model.show_parameters()
    print(f'------------------------------ {model_name} metrics ------------------------------')
    this_model.evaluate(args)
    print('')
    
    
    #pruning the model
    print(f'------------------------------ pruned {model_name} ------------------------------')
    pruning_model=copy.deepcopy(this_model)
    pruning_model.pruning(0.5)
    pruning_model.train({'output_dir':OUT_PATH, 'num_train_epochs':1})
    pruning_model.save_model('pruned')
    print(f'------------------------------ pruned {model_name} weights ------------------------------')
    pruning_model.show_parameters()
    print(f'------------------------------ pruned {model_name} metrics ------------------------------')
    pruning_model.evaluate(args)
    print('')
    
    
    # quantize of the model
    print(f'------------------------------ half {model_name} ------------------------------')
    half_model=copy.deepcopy(this_model)
    half_model.half()
    half_model.save_model('half')
    print(f'------------------------------ half {model_name} weights ------------------------------')
    half_model.show_parameters()
    print(f'------------------------------ half {model_name} metrics ------------------------------')
    half_model.evaluate(args)
    print('')
    
    
#     print(f'------------------------------ distiled {model_name} ------------------------------')
#     teacher_model = copy.deepcopy(this_model)
#     teacher_model.model.to(device).eval()
#     student_model = OurAwesomeModel('distilbert-base-uncased',dataset)    
#     train_dataset = teacher_model.dataset['train']
#     tokenizer = student_model.tokenizer
#     # Tokenize and encode the text data
#     train_dataset = train_dataset.map(lambda example: tokenizer(example['text'], truncation=True, padding='max_length'), batched=True)
#     # Convert the labels to integers
#     train_dataset = train_dataset.map(lambda example: {'labels': example['labels']}, batched=True)
#     student_model.model = knowladge_distilation_training(student_model.model.to(device), teacher_model.model.to(device), train_dataset)
#     student_model.save_model('student')
#     print(f'------------------------------ student {model_name} weights ------------------------------')
#     student_model.show_parameters()
#     print(f'------------------------------ student {model_name} metrics ------------------------------')
#     student_model.evaluate(args)
    
    
        
    models[model_name]= {'trained model': this_model, 'pruned_model':pruning_model,'half_model':half_model, 'student_model':student_model} 


============================== xlnet-base-cased ==============================


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

------------------------------ regular xlnet-base-cased ------------------------------
model was loaded
------------------------------ xlnet-base-cased weights ------------------------------
+-------------------------------------------------+------------+---------------+
|                     Modules                     | Parameters | Sum of Tensor |
+-------------------------------------------------+------------+---------------+
|               transformer.mask_emb              |    768     |     7.254     |
|        transformer.word_embedding.weight        |  24576000  |   1073323.0   |
|          transformer.layer.0.rel_attn.q         |   589824   |   44516.6953  |
|          transformer.layer.0.rel_attn.k         |   589824   |    94528.0    |
|          transformer.layer.0.rel_attn.v         |   589824   |   15092.6914  |
|          transformer.layer.0.rel_attn.o         |   589824   |   16119.0615  |
|          transformer.layer.0.rel_attn.r         |   589824   |   10848.5449  |

f1 value: 0.7080168983557641
accuracy value: 0.7100554578098796

------------------------------ pruned xlnet-base-cased ------------------------------

transformer.layer.0.ff.layer_1:
old_total_weights = -103.30775451660156
new_total_weights = -91.01393127441406

transformer.layer.0.ff.layer_2:
old_total_weights = -84.21345520019531
new_total_weights = -85.44734191894531

transformer.layer.1.ff.layer_1:
old_total_weights = -572.9107666015625
new_total_weights = -556.9412841796875

transformer.layer.1.ff.layer_2:
old_total_weights = 48.340579986572266
new_total_weights = 62.087646484375

transformer.layer.2.ff.layer_1:
old_total_weights = -391.30279541015625
new_total_weights = -354.6177978515625

transformer.layer.2.ff.layer_2:
old_total_weights = -18.800872802734375
new_total_weights = -9.930244445800781

transformer.layer.3.ff.layer_1:
old_total_weights = 245.47509765625
new_total_weights = 186.15032958984375

transformer.layer.3.ff.layer_2:
old_total_weights = -102.96053314208984
ne

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.738300
1000,0.751900
1500,0.812400
2000,0.775200
2500,0.788100
3000,0.791100
3500,0.818300
4000,0.799200
4500,0.790300
5000,0.787100


model was saved
------------------------------ pruned bert-base-uncased weights ------------------------------
+---------------------------------------------------------+------------+---------------+
|                         Modules                         | Parameters | Sum of Tensor |
+---------------------------------------------------------+------------+---------------+
|          bert.embeddings.word_embeddings.weight         |  23440896  |   957280.625  |
|        bert.embeddings.position_embeddings.weight       |   393216   |   4589.5972   |
|       bert.embeddings.token_type_embeddings.weight      |    1536    |    14.4349    |
|             bert.embeddings.LayerNorm.weight            |    768     |    652.8584   |
|              bert.embeddings.LayerNorm.bias             |    768     |    31.8757    |
|      bert.encoder.layer.0.attention.self.query.bias     |    768     |    167.1329   |
|     bert.encoder.layer.0.attention.self.query.weight    |   589824   |   17511.2344  |

f1 value: 0.7393312876568089
accuracy value: 0.7400582358448827

------------------------------ half bert-base-uncased ------------------------------
model was saved
------------------------------ half bert-base-uncased weights ------------------------------
+---------------------------------------------------------+------------+---------------+
|                         Modules                         | Parameters | Sum of Tensor |
+---------------------------------------------------------+------------+---------------+
|          bert.embeddings.word_embeddings.weight         |  23440896  |      inf      |
|        bert.embeddings.position_embeddings.weight       |   393216   |     4576.0    |
|       bert.embeddings.token_type_embeddings.weight      |    1536    |    13.9219    |
|             bert.embeddings.LayerNorm.weight            |    768     |     652.5     |
|              bert.embeddings.LayerNorm.bias             |    768     |    31.7344    |
|     bert.encoder.layer.0.at

f1 value: 0.733601302532763
accuracy value: 0.7348519924684385

